In [1]:
# coding: utf-8
import warnings
warnings.filterwarnings('ignore')
import time
import os
import numpy as np
import keras
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Flatten, Conv2D, Conv1D, MaxPooling1D, MaxPooling2D,Activation
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec
from collections import *
import pandas as pd

Using TensorFlow backend.


In [2]:
#idenity paramaters
word_vec_len = 256

In [3]:
# get data
# get texts data
category2idx = {'AllTogether': 0, 'Baseball': 1, 'Boy-Girl': 2, 'C_chat':  3, 'CVS': 4,
                  'GameSale': 5, 'GetMarry': 6, 'Lifeismoney': 7, 'LoL': 8, 'MH': 9, 'MLB': 10, 'Mobilecomm': 11, 
                'movie': 12,'MuscleBeach':  13, 'NBA': 14,  'SENIORHIGH': 15, 'Stock': 16, 
                'Tennis': 17, 'Tos': 18, 'WomenTalk': 19}

train_df_sample = pd.read_pickle('train.pkl').sample(frac=1, random_state=123)

In [4]:
# print(type(train_df_sample), len(train_df_sample))
# print(train_df_sample.label)

In [59]:
train_texts = train_df_sample.values
label_list = train_df_sample.label

test_pickle_df = pd.read_pickle('test.pkl')
test_texts = test_pickle_df["text"].values
# print(len(test_texts), test_texts[0]) 

In [60]:
train_texts_list = []
for text in train_texts:
    train_texts_list.append(text[0])
# print(len(train_texts_list), train_texts_list[0])

In [61]:
# get word embedding vector
answer = word2vec.Word2Vec.load("word2vec_20180430.model")
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)

In [62]:
# print(len(wvv))

In [63]:
texts_list = []
for text in train_texts_list:
#     print(text)
    texts_list.append(text)
    
for text in test_texts:
    texts_list.append(text)

In [10]:
print(type(texts_list), len(texts_list))
# print(label_list[0:10])

<class 'list'> 40000


In [11]:
del word_vectors, wvv, train_texts_list
# print(texts_list[0], "\n")
# print(texts_list[1], "\n")
# print(texts_list[2], "\n")

In [52]:
tokenizer1 = Tokenizer(num_words = 20000)
tokenizer1.fit_on_texts(texts_list)
sequences = tokenizer1.texts_to_sequences(texts_list)  # word index list
max_doc_word_length = 200
sequences1 = pad_sequences(sequences, maxlen=max_doc_word_length, padding='post')

print("Found %s unique tokens" % len(tokenizer1.word_index))

Found 571338 unique tokens


In [53]:
print(word_index["巧合"])
num_words = max(max(sequences))
print(num_words)
# print(max(sequences), sequences[1])

5619
19991
[5, 2, 3, 103]


In [14]:
# print(type(tokenizer1.word_index) , len(tokenizer1.word_index)) 
# print(len(sequences1), tokenizer1.num_words) 
# # print(tokenizer1.word_index)
# ss = 0
# for k in tokenizer1.word_docs:
#     if tokenizer1.word_docs[k] > 100:
#         print(k, tokenizer1.word_docs[k])
#         ss += 1
# print(ss)

In [15]:
# ss = 0
# for k in tokenizer1.word_docs:
#     if tokenizer1.word_docs[k] < 3 :
# #         print(k, tokenizer1.word_docs[k])
#         del word_index[k]
#         ss += 1
# print(ss)

In [73]:
%%time
# user 46min 33s,
vocab_size = num_words + 1
print(vocab_size, word_vec_len)
# create a weight matrix for wordds in training docs
embedding_matrix = np.zeros((vocab_size, word_vec_len))
ss = 0
word_index = tokenizer1.word_index
for word, i in word_index.items():
     if word in wvv_keys_list:
        embedding_vector = answer[word]
    #         if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.zeros((1, word_vec_len))   

19992 256
CPU times: user 51.7 s, sys: 80 ms, total: 51.7 s
Wall time: 51.7 s


In [21]:
del answer

In [22]:
# # network model
print(embedding_matrix.shape, len(embedding_matrix[10]))
# (571339, 256) 256

(571339, 256) 256


In [74]:
# image_width = 250
# image_height = 3
# input_channel = 1

embedding_layer = Embedding(
                            input_dim= embedding_matrix.shape[0],
                            output_dim= 256,
                            weights=[embedding_matrix], 
                            input_length = 200,
                            trainable=False)
# model = getModel(embedding_layer, image_width, image_height, input_channel)
model = Sequential()
model.add(embedding_layer )
# model.add(Reshape(4, 2))
# conv2D = 
# model.add(Conv1D(64, 3, 
#             border_mode='same',
#             input_shape=(3, 256)))
# model.output_shape == (None, 64, 32, 32)

# model.add(Flatten())
model.add(Conv1D(256, 3,
#                 filters=(9), 
#                 kernel_size = (256),
                padding = 'same', 
#                  input_shape=(9, 32, 32),
#                  input_shape=(10, 32)
                )
         )
# model.add(MaxPooling1D(pool_size=2))    
model.add(Flatten())
model.add(Dense(2, activation='relu'))
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=0, verbose=0, mode='auto')
model.compile(optimizer='adam',loss='categorical_crossentropy',  metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 200, 256)          5117952   
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 200, 256)          196864    
_________________________________________________________________
flatten_9 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 102402    
Total params: 5,417,218
Trainable params: 299,266
Non-trainable params: 5,117,952
_________________________________________________________________


In [67]:
# sequences2 = np.reshape(sequences1,(2, 20000, 200)) 
print(sequences1.shape, label_list.shape, label_list[0])

Y_label_list = np.zeros((36000, 1, 2))
for ids in range(0, 36000):
    Y_label_list[ids][0][0] = label_list[ids][0]
    Y_label_list[ids][0][1] = label_list[ids][1]
print(Y_label_list.shape)

(40000, 200) (36000,) ['11', '0']
(36000, 1, 2)


In [71]:
history = model.fit(x = sequences1[0:100], 
                    y = Y_label_list[0:100], 
                    validation_split=0.1, 
                    batch_size= 18000,
                    epochs = 100, verbose = 1)

ValueError: Error when checking input: expected embedding_12_input to have shape (200,) but got array with shape (1,)

In [ ]:
# model.save('my_model.h5') 

In [ ]:
# evaluate the model
loss_accuracy = model.evaluate(sequences1[0:100], label_list[0:100], verbose=1)
print(type(loss_accuracy), loss_accuracy)

In [ ]:
test_sequences1 = sequences1[9000:10000]

print(len(sequences1))
print(len(test_sequences1))

In [ ]:
predict_res = model.predict(test_sequences1, batch_size= 32, verbose=0)
print(len(predict_res), predict_res)

final_res = []
for pre_res in predict_res:
    final_res.append(np.argmax(pre_res))
print(final_res)

In [ ]:
# result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
print(len(final_res))
result_txt = "server_result001" + ".txt"
ids = 0
with open(result_txt, 'w') as out:
    out.write("id,category" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(value) + '\n')
        ids += 1